In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 1. Imports
# ─────────────────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import calendar
import ipywidgets as w
from IPython.display import display, HTML

from worldcereal.utils.refdata import month_diff, get_best_valid_time

HTML("""
<style>
  .widget-label        { font-size:18px !important; white-space:normal !important; }
  .widget-readout      { font-size:18px !important; }
  .widget-slider .ui-slider-handle { font-size:18px !important; }
</style>
""")

# ─────────────────────────────────────────────────────────────────────────────
# 2. Helper function
# ─────────────────────────────────────────────────────────────────────────────
def evaluate(start_date, end_date, valid_time, buffer, num_ts=12):
    row = pd.Series({"start_date": start_date, "end_date": end_date,
                     "valid_time": valid_time})
    res = []
    for m in range(1, 13):
        row["true_valid_time_month"] = valid_time.month
        row["proposed_valid_time_month"] = m
        row["valid_month_shift_backward"] = month_diff(m, row["true_valid_time_month"])
        row["valid_month_shift_forward"]  = month_diff(row["true_valid_time_month"], m)
        res.append([m, get_best_valid_time(row, buffer, num_ts)])

    df = pd.DataFrame(res, columns=["proposed_month", "resulting_valid_time"])
    df["proposed_month_str"] = df["proposed_month"].map(calendar.month_abbr.__getitem__)
    df["acceptable"] = df["resulting_valid_time"].notna()
    return df

# ─────────────────────────────────────────────────────────────────────────────
# 3. Helper to build a DatePicker + ‹/› buttons
# ─────────────────────────────────────────────────────────────────────────────
def date_picker_with_arrows(label_text, init_date):
    """Return (HBox, datepicker) with label – left – datepicker – right."""
    label = w.Label(value=label_text, layout=w.Layout(width="160px"))  # adjust width if needed
    left  = w.Button(icon="chevron-left",  layout=w.Layout(width="32px"))
    right = w.Button(icon="chevron-right", layout=w.Layout(width="32px"))
    dp    = w.DatePicker(value=init_date, description="", layout=w.Layout(width="140px"))

    def shift(months):
        if dp.value:
            dp.value = (pd.Timestamp(dp.value) + pd.DateOffset(months=months)).to_pydatetime()

    left.on_click(lambda _: shift(-1))
    right.on_click(lambda _: shift(1))

    # order: label → left arrow → datepicker → right arrow
    return w.HBox([label, left, dp, right], layout=w.Layout(align_items="center")), dp

# ─────────────────────────────────────────────────────────────────────────────
# 3. Widgets
# ─────────────────────────────────────────────────────────────────────────────
start_box, start_w = date_picker_with_arrows("Extractions start date", pd.Timestamp("2018-08-01"))
end_box,   end_w   = date_picker_with_arrows("Extractions end date",  pd.Timestamp("2019-11-30"))
valid_box, valid_w = date_picker_with_arrows("True valid time",       pd.Timestamp("2019-06-01"))

buffer_w = w.IntSlider(value=2, min=0, max=6, step=1,
                       description="Buffer (months)",
                       style={"description_width":"initial"},
                       layout=w.Layout(width="300px"))

buffer_note = w.HTML(
    "<i>How close (in months) we allow the true valid_time to be to the edges of the user-defined temporal extent (TE).</i>"
)

# ─────────────────────────────────────────────────────────────────────────────
# 4. Visual grouping
# ─────────────────────────────────────────────────────────────────────────────
frame_title = w.HTML("<b>Hypothetical sample from a public dataset: select available extractions start and end dates and valid time (use ‹/› to jump a month)</b>")
date_box = w.VBox([frame_title, start_box, end_box, valid_box],
                  layout=w.Layout(border="1px solid #ccc",
                                  padding="10px", margin="5px 0"))
dates_frame = w.VBox([date_box])

buffer_frame_title = w.HTML("<b>Buffer settings</b>")
buffer_box = w.VBox([buffer_w, buffer_note])
buffer_frame = w.VBox([buffer_frame_title, buffer_box],
                      layout=w.Layout(border="1px solid #ccc",
                                      padding="10px", margin="5px 0"))

# ─────────────────────────────────────────────────────────────────────────────
# 5. Reactive area
# ─────────────────────────────────────────────────────────────────────────────
warn_style = "color:red; font-weight:bold; white-space:pre-line"

def update_view(start_date, end_date, valid_time, buffer):
    """Either show the styled table or a red warning."""
    msgs = []
    if start_date is None or end_date is None or valid_time is None:
        msgs.append("All three dates must be set.")
    else:
        if end_date <= start_date:
            msgs.append("The <b>End</b> date must be <u>after</u> the <b>Start</b> date.")
        if not (pd.to_datetime(start_date) < pd.to_datetime(valid_time) < pd.to_datetime(end_date)):
            msgs.append("The <b>valid_time</b> must lie <u>strictly inside</u> the chosen period.")

    if msgs:
        warning_html = f"<div style='{warn_style}'>" + "<br>".join(msgs) + "</div>"
        display(w.HTML(warning_html))
        return

    # if okay → build table
    df = evaluate(pd.to_datetime(start_date), pd.to_datetime(end_date), pd.to_datetime(valid_time), buffer)
    df_disp = (df[["proposed_month_str","acceptable"]]
               .rename(columns={"proposed_month_str":"Middle month\nof user-defined TE",
                                "acceptable":"Will the sample\nbe used?"}))
    def red(r): return ["background-color:#fdd" if not r["Will the sample\nbe used?"] else "" for _ in r]
    styler=(df_disp.style
            .hide(axis="index")
            .apply(red,axis=1)
            .format({"Will the sample\nbe used?": lambda x:"✓" if x else "DISMISSED"})
            .set_table_styles([
                {"selector":"th","props":[("text-align","center"),("white-space","pre-line"),("font-size","15px")]},
                {"selector":"td","props":[("text-align","center"),("white-space","pre-line"),("font-size","15px")]}
            ]))
    display(styler)

table_output = w.interactive_output(
    update_view,
    {"start_date": start_w, "end_date": end_w,
     "valid_time": valid_w, "buffer": buffer_w}
)
table_output.layout = w.Layout(min_height="320px")

# ─────────────────────────────────────────────────────────────────────────────
# 6. Assemble and show the full UI
# ─────────────────────────────────────────────────────────────────────────────
main_title = w.HTML("<h2>Temporal shift checker for sample acceptability</h2>")
table_title = w.HTML("<h3>Which user-defined temporal extents (TE) will be aligned with the sample:</h3>")
display(w.VBox([main_title,
                dates_frame,
                buffer_frame,
                table_title,
                table_output]))
